# Zig-zag
[Zig-zag scan](https://en.wikipedia.org/wiki/JPEG#Entropy_coding) of a matrix is useful when you transform your 2D data between domains and the transformation tends to accumulate information into top left corner.

An example is [JPEG](https://en.wikipedia.org/wiki/JPEG) image compression with [discrete cosine transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform), which accumulates energy to the left [in 1D] and top [in 2D].

![ZigZag 1](resource/day63-zigzag_1.png)

To approach the problem, it’s best to look at the matrix and notice some numerical rules.

![ZigZag 2](resource/day63-zigzag_2.png)


* indices `[i, j]` at each consecutive scan sum up to the same value, e.g `[0, 3] [1, 2] [2, 1] [3, 0]` sum up to `3`
* the direction of a scan alters between odd and even positions of the first item `[i, 0]`
* scan at position `[i, 0]` is preceded by `1+2+…+i = i*(i+1)/2` items
* when the matrix is split at minor diagonal, the values in top-left and bottom-right triangles follow the equation
* bottom-right`[i, j] == n²-1 — top-left[n-1-i, n-1-j]`

Now we have enough information to write both conversions, from value to index and from index to value.

In [1]:
import numpy as np

## algorithm

In [2]:
def zig_zag_index(k, n):
    # upper side of interval
    if k >= n * (n + 1) // 2:
        i, j = zig_zag_index(n * n - 1 - k, n)
        return n - 1 - i, n - 1 - j

    # lower side of interval
    i = int((np.sqrt(1 + 8 * k) - 1) / 2)
    j = k - i * (i + 1) // 2
    return (j, i - j) if i & 1 else (i - j, j)

In [3]:
def zig_zag_value(i, j, n):
    # upper side of interval
    if i + j >= n:
        return n * n - 1 - zig_zag_value(n - 1 - i, n - 1 - j, n)

    # lower side of interval
    k = (i + j) * (i + j + 1) // 2
    return k + i if (i + j) & 1 else k + j

## run

In [4]:
n = 10

In [5]:
M = np.zeros((n, n), dtype=int)
for i in range(n):
    for j in range(n):
        M[i, j] = zig_zag_value(i, j, n)
M

array([[ 0,  1,  5,  6, 14, 15, 27, 28, 44, 45],
       [ 2,  4,  7, 13, 16, 26, 29, 43, 46, 63],
       [ 3,  8, 12, 17, 25, 30, 42, 47, 62, 64],
       [ 9, 11, 18, 24, 31, 41, 48, 61, 65, 78],
       [10, 19, 23, 32, 40, 49, 60, 66, 77, 79],
       [20, 22, 33, 39, 50, 59, 67, 76, 80, 89],
       [21, 34, 38, 51, 58, 68, 75, 81, 88, 90],
       [35, 37, 52, 57, 69, 74, 82, 87, 91, 96],
       [36, 53, 56, 70, 73, 83, 86, 92, 95, 97],
       [54, 55, 71, 72, 84, 85, 93, 94, 98, 99]])

In [6]:
M = np.zeros((n, n), dtype=int)
for k in range(n * n):
    M[zig_zag_index(k, n)] = k
M

array([[ 0,  1,  5,  6, 14, 15, 27, 28, 44, 45],
       [ 2,  4,  7, 13, 16, 26, 29, 43, 46, 63],
       [ 3,  8, 12, 17, 25, 30, 42, 47, 62, 64],
       [ 9, 11, 18, 24, 31, 41, 48, 61, 65, 78],
       [10, 19, 23, 32, 40, 49, 60, 66, 77, 79],
       [20, 22, 33, 39, 50, 59, 67, 76, 80, 89],
       [21, 34, 38, 51, 58, 68, 75, 81, 88, 90],
       [35, 37, 52, 57, 69, 74, 82, 87, 91, 96],
       [36, 53, 56, 70, 73, 83, 86, 92, 95, 97],
       [54, 55, 71, 72, 84, 85, 93, 94, 98, 99]])